In [2]:
using Random
using Printf
using Dates

In [3]:
# convert the msa to tsp graph. Takes sequences and input and returns the distance matrix

function MSA_to_TSP(sequences)
  node = []
  for i in 1:length(sequences)
    push!(node,sequences[i])
  end

  graph = Array{Float64, 2}(undef, length(node), length(node))
  max_score = 0
  for i in 1:length(node)
    graph[i,i] = 0
    for j in i+1:length(node)
      score, align1, align2 = produce_global_alignment(node[i],node[j]) 
      graph[i,j] = score
      graph[j,i] = score
      if max_score < score 
        max_score = score
      end
    end
  end
    
  # convert score to distance, more the score lesser the distance
  for i in 1:length(node)
    for j in i+1:length(node)
      graph[i,j] = max_score - graph[i,j] + 1
      graph[j,i] = max_score - graph[j,i] + 1
    end
  end    
  return graph
end

MSA_to_TSP (generic function with 1 method)

In [4]:
# perform the needleman-wunsch on two sequences v and w, returns the score and algined sequences

function produce_global_alignment(v, w, match_penalty=1, mismatch_penalty=-1, deletion_penalty=-1)
    rows = length(v)+1
    cols = length(w)+1
    dp = zeros(Float64, rows, cols)
    direction = zeros(Float64, rows, cols)
    for i in 1:(rows)
        dp[i,1] = (i-1) * deletion_penalty
        direction[i,1] = 2
    end
    for i in 1:(cols)
        dp[1,i] = (i-1) * deletion_penalty
        direction[1,i] = 3
    end
    for i in 2:(rows)
        for j in 2:(cols)
            if v[i-1] == w[j-1]
                ms = dp[i-1,j-1] + match_penalty
            else
                ms = dp[i-1,j-1] + mismatch_penalty
            end
            score = [ms, dp[i-1,j] + deletion_penalty, dp[i,j-1] + deletion_penalty]
            max_element = findmax(score)
            dp[i,j] = max_element[1]
            direction[i,j] = max_element[2]
        end
    end
    i = rows
    j = cols
    aligned_v = []
    aligned_w = []
    while(i > 1 || j > 1)
        dir = direction[i,j]
        if dir == 1
            i = i-1
            j = j-1
            push!(aligned_v, v[i])
            push!(aligned_w, w[j])
        end
        if dir == 2
            i=i-1
            push!(aligned_v, v[i])
            push!(aligned_w, "-")
        end
        if dir == 3
            j = j-1
            push!(aligned_v, "-")
            push!(aligned_w, w[j])
        end
    end
    return (dp[rows,cols], join(reverse(aligned_v)), join(reverse(aligned_w)))
end

produce_global_alignment (generic function with 4 methods)

In [5]:
# returns the ant colony with cities initialized to each ant

function create_colony(num_ants, num_nodes)
    colony = []
    for i in 1 : num_ants
       push!(colony, Dict("path"=>[rand(1:num_nodes)], "distance" => 0))
    end
    return colony
end

# create the pheror matrix
function create_pheror_matrix(num_nodes)
    pheromone = zeros(Float64, num_nodes, num_nodes)
    for i in 1: num_nodes
        for j in 1: num_nodes
            pheromone[i,j] = 1/num_nodes
        end
    end
    return pheromone
end

create_pheror_matrix (generic function with 1 method)

In [6]:
# intialize the probability matrix

function calculate_proba(num_nodes, pheromone, distance_matrix, alpha, beta)
    probability = zeros(Float64, num_nodes, num_nodes)
    for i in 1: num_nodes
        for j in 1: num_nodes
            probability[i,j] = (pheromone[i,j]^alpha) * (distance_matrix[i,j]^-beta)
            probability[j,i] = probability[i,j]
        end
    end
    return probability
end

calculate_proba (generic function with 1 method)

In [7]:
# calculate probability of each ant going to all other nodes from current node

function calculate_proba_ant(pheromone, distance_matrix, unvisited_nodes, current_node, proba, alpha, beta)
  sigma = 0.0
  for unvisited_node in unvisited_nodes
    sigma += (pheromone[current_node,unvisited_node]^alpha) * (distance_matrix[current_node,unvisited_node]^-beta)
  end
  proba_ant = proba[current_node,:]/sigma
  return proba_ant
end

calculate_proba_ant (generic function with 1 method)

In [8]:
# finds the best path out of all ants

function find_best_path(n_ants, colony)
  bpath = []
  best_distance = Inf32
  typeof(best_distance)
  idx_best = 0
  for i=1: n_ants
    if colony[i]["distance"] < best_distance
      best_distance = colony[i]["distance"]
      bpath = colony[i]["path"]
      idx_best = i
    end
  end
  best_path = Dict("path"=> bpath, "distance"=> best_distance, "ant"=> idx_best)
  return best_path
end

find_best_path (generic function with 1 method)

In [9]:
# updates the pheror matrix

function update_pheror_matrix(num_nodes, n_ants, pheromone, distance_matrix, colony, Q, decay)
  depositpher = 0.0
  for i=1: n_ants
    ant = i
    for j= 1:(length(colony[ant]["path"])-1)
      src = colony[ant]["path"][j]
      dest = colony[ant]["path"][j+1]
      pheromone[src,dest] += Q/colony[i]["distance"]
    end
    depositpher += Q/colony[i]["distance"]
    for i= 1:num_nodes
      for j= 1:num_nodes
        pheromone[i,j] = (1-decay)*pheromone[i,j]*depositpher
        pheromone[j,i] = pheromone[i,j]
      end
    end
  end
  return pheromone
end

update_pheror_matrix (generic function with 1 method)

In [10]:
# calculate distance for one ant

function calculateDist_ant(ant, colony, distmatrix)
  dist = 0
  path = colony[ant]["path"]
  for i= 1:length(path)-1
    dist += distmatrix[path[i],path[i+1]]
  end
  return dist
end

calculateDist_ant (generic function with 1 method)

In [11]:
# run each ant from source to destination

function traverse(ant, num_nodes, colony, pheromone, distance_matrix, proba, alpha, beta)
    unvisited = collect(1:num_nodes)
    current = colony[ant]["path"][1]
    deleteat!(unvisited, findfirst(isequal(current), unvisited))
    for j in 1: num_nodes-1
        if length(unvisited) > 1
            ant_probability = calculate_proba_ant(pheromone, distance_matrix, unvisited, current, proba, alpha, beta)
            prob = map((x) -> ant_probability[x] , unvisited)
            current = unvisited[findmax(prob)[2]]
            deleteat!(unvisited, findfirst(isequal(current), unvisited))
            push!(colony[ant]["path"], current)       
        else
            push!(colony[ant]["path"], unvisited[1])
        end
    end
    colony[ant]["distance"] = calculateDist_ant(ant, colony, distance_matrix)
end

traverse (generic function with 1 method)

In [12]:
# driver function for aco

function aco_main(num_ants, num_nodes, distance_matrix, iterations, Q, decay, alpha, beta)
    pheromone = create_pheror_matrix(num_nodes)
    gbpath = Dict()
    for i= 1: iterations
        colony = create_colony(num_ants, num_nodes)
        probability = calculate_proba(num_nodes, pheromone, distance_matrix, alpha, beta)
        for ant in 1: num_ants
            traverse(ant, num_nodes, colony, pheromone, distance_matrix, probability, alpha, beta)
        end
        # complete update_pheromone_matrix
        pheromone = update_pheror_matrix(num_nodes, num_ants, pheromone, distance_matrix, colony, Q, decay)
        #complete find best path fucntion
        best_path = find_best_path(num_ants, colony)
        
        bpath = best_path
        if i == 1
            gbpath = bpath
        else
            if bpath["distance"] < gbpath["distance"]
                gbpath = bpath
            end
        end
    end 
    # return the best path
    return gbpath["path"]
end

aco_main (generic function with 1 method)

In [13]:
function find_gap_indices(A, alignedA)
    i = 1
    j = 1
    pointer = []
    while j <= length(alignedA)
        if alignedA[j] == '-' && (i > length(A) || A[i] != '-')
            push!(pointer, j)
            j += 1
        else
            j += 1
            i += 1
        end
    end
    return pointer
end

find_gap_indices (generic function with 1 method)

In [14]:
function insert_gaps(S,gap_indices_for_A)
    copy_of_S = S
    if length(gap_indices_for_A) > 0 && length(gap_indices_for_A) > 0
        gap_indices_for_A = sort(gap_indices_for_A)
        for i in gap_indices_for_A
            copy_of_S = string(string(copy_of_S[1:i-1],'-'),copy_of_S[i:end])
        end
    end
    return copy_of_S
end

insert_gaps (generic function with 1 method)

In [15]:
# this function takes two params: sequences is the original array of input sequences, order is the array output of tsp algorithm of the order of sequences
# notice that the index in both the params are relative to each other

function align_output_sequences(sequences, order) 
    ordered_sequences = Array{String,1}(undef,0)
    for i=1:length(order)
        push!(ordered_sequences,sequences[order[i]])
    end
    aligned_sequences = Array{String,1}(undef,0)
    for i=1:length(ordered_sequences)
        push!(aligned_sequences,ordered_sequences[i])
    end
    for i=1:length(aligned_sequences)-1
        A = aligned_sequences[i]
        B = aligned_sequences[i+1]
        score, alignedA, alignedB = produce_global_alignment(A,B)
        gap_indices_for_A = find_gap_indices(A, alignedA)
        # go to all predecessors of A and insert the gaps at same place
        for j = 1:i-1
            S = aligned_sequences[j]
            newly_alinged_S = insert_gaps(S,gap_indices_for_A)
            aligned_sequences[j] = newly_alinged_S
        end
        aligned_sequences[i] = alignedA
        aligned_sequences[i+1] = alignedB
    end  
    ordered_aligned_sequences = Array{String,1}(undef,length(order))
    for i=1:length(order)
        ordered_aligned_sequences[order[i]] = aligned_sequences[i]
    end
    return ordered_aligned_sequences
end

align_output_sequences (generic function with 1 method)

In [16]:
# calculate the final score

function calc_sum_pair(sequences) 
    t = length(sequences)
    k = length(sequences[1])
    score = 0
    for i=1:t
        A = sequences[i]
        for j=i+1:t
            B = sequences[j]
            for idx = 1:k
                if A[idx] == B[idx] && A[idx] != '-'
                    score += 1
                end
            end
        end
    end
    return score
end

calc_sum_pair (generic function with 1 method)

In [17]:
# calculate the final score to include gap penalty

function calc_sum_pair_include_gap_penalty(sequences) 
    t = length(sequences)
    k = length(sequences[1])
    score = 0
    for i=1:t
        A = sequences[i]
        for j=i+1:t
            B = sequences[j]
            for idx = 1:k
                # Add 1 for match
                if A[idx] == B[idx] && A[idx] != '-'
                    score += 1
                # subtract 1 for mismatch
                elseif A[idx] != B[idx] && A[idx] != '-' && B[idx] != '-'
                    score -= 1
                # subtract 1 for gap
                elseif A[idx] == '-' ||  B[idx] == '-' 
                    score -= 1
                end
            end
        end
    end
    return score
end

calc_sum_pair_include_gap_penalty (generic function with 1 method)

In [18]:
# get sequences from the input file

function get_sequences_from_file(file_name)
    sequences = []
    flag_first_arrow = true
    open(file_name) do f
        sequence = ""
        for line in eachline(f)
            if startswith(line, '>') 
                if flag_first_arrow
                    flag_first_arrow = false
                    continue
                end
                push!(sequences, sequence)
                sequence = ""
            else
                sequence *= line
            end
        end
        if length(sequence) > 0
            push!(sequences, sequence)
        end
    end
    return sequences
end

get_sequences_from_file (generic function with 1 method)

In [19]:
# write output to the file

function write_to_file(output_filename, aligned_sequences)
    touch(output_filename)
    f = open(output_filename, "w") 
    for i=1:length(aligned_sequences)
        seq_num = string(">s",i)
        write(f,string(seq_num,"\n"))
        write(f,aligned_sequences[i])
        write(f,"\n")
    end
    close(f) 
end

write_to_file (generic function with 1 method)

In [22]:
# this is the driver function which take the fasta file as input and writes the output to the output.txt file.
# the default input to this function is input.fasta which should be present in the same folder as this code.

function driver(filename="dataset.txt",output_filename = "output1.txt")
    # get the sequences from input file
    input_sequences = get_sequences_from_file(filename)
    # convert the get input_sequences to TSP problem and get the distance_matrix for TSP
    input_distance_matrix = MSA_to_TSP(input_sequences)
    # configuration for ACO
    num_sequences = length(input_sequences)
    num_ants = maximum([100,5*num_sequences])
    iterations = 50
    Q = 0.6
    decay = 0.6
    alpha = 1
    beta = 1
    # run the ant colony optimization, get the order of node traversal
    order_of_alignment = aco_main(num_ants,num_sequences,input_distance_matrix,iterations,Q,decay,alpha,beta)
    # align output sequences in the order identifiedgi by above algorithm
    aligned_sequences = align_output_sequences(input_sequences,order_of_alignment)
    # get the score of the aligned_sequences
    final_score = calc_sum_pair(aligned_sequences)
    # save the output in fasta format in output1.txt
    write_to_file(output_filename, aligned_sequences)
    # print the aligned sequences
    println("Score of aligned sequences = ",final_score)
    println("The aligned sequences are saved in output file. Printing them here as well:")
    for i=1:length(aligned_sequences)
        println(aligned_sequences[i])
    end  
    return final_score, aligned_sequences
end

driver (generic function with 3 methods)

In [ ]:
# driver()

In [24]:
using Plots

In [25]:
function generator(sequence_length, number_of_sequence)
    seuqences = []
    bases = ["A", "C", "G", "T"]
    current_sequence = []
    for i in 1:sequence_length
        push!(current_sequence, bases[rand(1:4)])
    end
    push!(seuqences, join(current_sequence))
    for i in 1:number_of_sequence-1
        changes = rand(floor(0.3*sequence_length) : floor(0.6*sequence_length))
        for t in 1: changes
            location = rand(1:sequence_length)
            current_sequence[location] = bases[rand(1:4)]
        end
        push!(seuqences, join(current_sequence))
    end
    return seuqences
end

generator (generic function with 1 method)

In [76]:
function score_ACO_vs_GA()
    scores = []
    for i in 1:10
        input_file = "input"*string(i)*".fasta"
        output = "score"*string(i)*".fasta"
        input_sequences = generator(50*i, 10)
        write_to_file(input_file, input_sequences)
        score, result = driver(input_file, output)
        push!(scores, score)
    end
    return scores
end

score_ACO_vs_GA (generic function with 1 method)

In [77]:
# get ACO scores

scores = score_ACO_vs_GA()

Score of aligned sequences = 1008
The aligned sequences are saved in output file. Printing them here as well:
GAC--TAG--C-CTAAACC-CTCCCTGGGCTA--A-GC-AATCTACTGCA--TCCTAC-CAAA
TACT-TGGTTC---GATCCGC-C-CTGGGCTA--A-GCG-CTCTACTGCA-GT-CTAC-CAAT
TAAT-TTCTTC---GATACGCACG--GGGCTA--ATT-G-CTCTACTGTA-GG-CT-CGCAAT
TGAT-TTGTTC-G-G-TCCCCACGAAG--CTA--ATTGG--TATGCTGTA-GA-CA-CGCAAT
CGAT-TT-TTCTG-GTTGCCT-CG--G-ACCA-TATTGG--TACGCGGAA-GG-CA-AACGAT
CTCT-AT-TTCTGA-TTG--TG-GT-GTTCCAG-AATGGTTTAC-C-GAACGG--A-AACGGT
C-CTCAT-TACAGA-TTG--TG-CT-CCTCCAGCAA-GCTTTAC-C-GAACGA--A-AACTTT
C-CTCCG-TACAGA-TTT--TGA-T-CCTGAAGCAA-GCTTTGC-C-TCACGA--A-AACCTC
C-CGCCC-TACA-ACACT--T-ACT-CGTGAAGCAA-GCTTTGC-C-TCACGA--A-AACCGC
C-CGCGC-GACA-ACAAT--T-GCT-AGCAAAGTAT-GATTTGT-C-TCACCA--A-AAGTGG
Score of aligned sequences = 2245
The aligned sequences are saved in output file. Printing them here as well:
-AA-AATGTGTCTGCGCA-GGCC--TCA-CGTACAAGG-G--CC-ACA-TC-GCT-AAGGCA-TCA-AGGGG-CCATCAC-CAG-ACAGAAAA-GT-TACTTAT--GATAC-CGCCTCGCTCT-
GAA-TA-GAGTCTGC

ACCAAGGTTA-T-TCCG-ATCCT--TACT--TCTGCCTCA----CCATAT-AAGTAC-CTC--T--GTTA-TCACTTCTG-T--GTTACCGT--AATCAG-GAATAAACT--TGGA--AACTGTAC-C-C-CCCG-ACT----CC-AT-TTTTTG--TGATAG-CG-C-GTT-TCACGGCAACGCAGCAT--AAAC-TTGGG-GTGA---CGCG-A--CGTCAAACTGGCTG----GA---TA-CA--T-A-T-TGCGT-T-TCCCTACATTACAC----C-----CAGGGATG-CAT-CAACG-TG-TCACTT-TTATACTGGTAAAT-ATCCTC-CAC-C-CAG----CAG-ATCG-CCGTCCG-AG--GGAAACTA-CAAT-GCATACC-GGCG-TC
AGCAATGCTA-TA-CCG-TTAGT--TACTC--CTGCCTTA----TCATTTC-AGCGC-TTC--T--GTTAC-CAGTTCTG-T--CTGAACTT--GACCCG-TAATACACT--TGGA--AAGTGTACTCAC-CG-G--CT----CCAAT-TGT-TG--AGATTG-TG-CGGTT--CGCGGCAACGCAGCAT--TAAC-TCGTG-GTGA---CG-GCA--CGTCAGCCTGGCTG----GAG---C-CA--TAAGT-CGC-T-TGT-CGT-CTTTACAC----CA-----ACGGATG-CTTA-AACG-TG-CCACTT-TTAGACTAGTAAGTG-GCCTCA-ACT--CTG----CGG-ATCG-CCGTTCG-AGC-GTTA-C-ACCATT-GTAT-CCAGCCG-TC
AGGATTGCCA-CA-CCG-TTA-TC-TACTCGGAT-A-TGAG----CATTTC-AGCGC-TTC--T--CTTAC-AAG---TGATAACTGAACTT--TTCCAG-T-ATA-ACTCATGGAC--A-T-TA-T-ACTTG-GCTCTC---CCCGT-TAT-CG--AGAGTG--GACTGTTA-C-CGGAAACGCCGCAC--AAAC-T

GCTTGGGGCGCGCCCTG-G--AT-GATACTACT-AG-CTG-ACCCGAG-A--CTCATCGC-GGCCA-CC--A--TCCAG-T-TCGG-TAA-AC-C-T--TAATATCACTC-GGACCTTGGCAAAG-G--GAAGAC-T-GGG-TAGGCC-C-G-CGC-GGCGGCCAC-TGGACGGGCA-ACA-GC--AC-G--CAGAGAA-CTCGCTTT-T-ATTG-GCTTACGTC-CGA-CGTGTGA-C--CCC-GG-CGGCGA-TT-CCT-T-GGATTA---T-TAATAGAAACG--AATAG-A-AGA-GTCTC--A-A-AT-G-TGAGACT-GAAGA-CTGATCCATGGAATATAGCCAC--TCAG-GCC-ATTGAC--CT-CTACAATACCGTAACT-GCG--AACTGGGTGTCACT-CCCAGCTGAC--GACAGCGTC--GC---CAGGCTA-TAC-TGTTACGT-TGTTCGA-TACCACTTC-T-GTCACAACA--GCAGTCT-GTC
GAATGTCGCGCGCCCAACG-TATGGA-A--AAT-AG-CT-TACCGGTG-A--CTCATCGC-AGCC-GCT--A--T-CTGGT-TCGG--AAGTC-CGT--T-ATATCACTC-GG-GCTTTG-ACAGCG-TTAAGACAT--TG-TTGGCCACTG-AGC---CGGTGACTTCG-CAGG--TAC---CA-AC-GTGCGGGGCA-CCCGCTGT-T-AATG-GGGTAAGAC-C-A-C-CGTGA-C-ACCCGGG-CGGCGAGTT--TT-T-GGAGT---GT-AAATAGAAACG--GACAG---AGGTGTTTT--A-A-TT-GAT-AGACT-GAA-TCCTGATCCACGGATTATAATCAC--TCAG-GCCTA-TGA--ATTGCCGC-ATACCCT---TAGAGACAACTGGGTGTC-GTTTCCATCTGAC-AG-CAGCGTC-TG----CAGCCTA-TGC-TGTTAAGG-CGTTCGA-TACCTCTTC--AGTCTCAGCA-CG-AGGG

A----TGTAGGA-AAAGC-TAAACCA--TATT-GGCACATC-CTTGCA-CC-A-TGCCTT-G-TA-G---T---TAGTAGCAGTCGT-TCGTGCATAAA-T-CTCGAGGA-GGA-TT-GACACTCAC-CTATGTGTTCG--GCA-GGGA--AT-TGT-TG-TGATGGCC--AATC--GCGAAT-TTTTAT-TACGC-GACCTG-C-CGACTATGTGA--CCTGCTA-CTGCG-AAAC--A-TGATG-CCTAC---TCCAA--GGG-CC-CATCCA-GAG--GTGA-GCACGGCGCGGTACGTT-GCA-ACG--AG-TCGC-AT-G-TCGCCCCTAGC-ACTTCCGTGCT-G-CCA-C-CAGAGT-TATAGGT-GTTTG-ACTGCCTGACC-G-GATA--TTCTT-C-GCGC--AT-GGCCG-ACCCA-C-CGTAA-AGCGTTGTGAGGATGG-GTGAGCGTTTGCCAGCGACAGG-T-CGGCA----AGG-GGA---AA-TT-CG--TGTGT-GGAGAAGTCAC-ATGTTC-A-CA-G-G-GTCCAAT-G-TTC-ACTAGG-TACATACACCGC-CAGGAGCC--CGT--AAGG--C-ACGTC-TGCATACTCCCATCCGCAT-GTACTG
A---G-GTAGGA-GAAGC-TAAA-GAG-TATT-GGTATAAC-CTTGGT-CC-A-TGCCTT-G-TA-G---T-G--ACTCGCAGTGGT-TCGTGC-CAAACT-GTCGAAGACG-A-TT-GACACACAC-CTATTTTTTTG--GCA-GGTA--AT-CGT-TG-TGATGGGG--GATC--GCGAAT-TTTTATCCATG--GAACTG-C-CTAATAAATGA--ACTGCTA-CTGAG-GTAC--A-TGCTGT-CTAC---TCCAAC-GGG--C-CTTCCA-GAG--GTTA-GCTCGGCGCCGTACGTT-GTA-AAG--AG-TCG--ATGG-TCCCCCCTAGC-ACTTCCGT-CTAG-CCA-C-AATAGT-TATA

10-element Vector{Any}:
  1008
  2245
  3381
  4263
  5416
  6561
  7594
  8717
  9673
 11333

In [79]:
# get GA scores

ga_scores = [];

f = open("GAscores.txt")
f = readlines(f)

for str in f
    str = parse(Float64, str)
    floor(str)
    push!(ga_scores, str)
end

In [80]:
ga_scores

10-element Vector{Any}:
  1322.0
  2461.0
  3598.0
  4736.0
  5877.0
  7016.0
  8155.0
  9295.0
 10435.0
 11575.0

In [83]:
function plot_score_ACO_vs_GA()
    x = collect(50:50:500)
    # different length sequences
    plot(x, [scores, ga_scores],
        title = "ACO Scores vs GA Scores for 10 sequences", 
        label = ["Score by ACO" "Score by GA"]
    )
    xlabel!("Length of Sequence")
    ylabel!("Scores")
    savefig("plot_score_ACO_vs_GA.png")
end

plot_score_ACO_vs_GA (generic function with 1 method)

In [84]:
plot_score_ACO_vs_GA()

In [85]:
function time_ACO_vs_GA()
    elapsed_times = []
    for i in 1:10
        input_file = "input"*string(i)*".fasta"
        output = "time"*string(i)*".fasta"
        input_sequences = generator(50*i, 10)
        write_to_file(input_file, input_sequences)
        result, elapsed_time, bytes_allocated, garbage_c_time, mac =  @timed driver(input_file, output)
        push!(elapsed_times, elapsed_time)
    end
    return elapsed_times
end

time_ACO_vs_GA (generic function with 1 method)

In [86]:
# get ACO scores

ACO_times = time_ACO_vs_GA()

Score of aligned sequences = 1175
The aligned sequences are saved in output file. Printing them here as well:
--TGCCCGCA--AGCTGCAATT-TTCTGTCGTGTTCGT-GTC-CGGATCA-AAGC-CGT
--CGCAGGCAT--CCAGCAATT-ATCTGTCAAGCTCGT-GTC-CGAATCACA-CC-CTT
-TAG-AGGCAT-A-CATCAATT-TTCTGTCAATCTCGT-GTC-CTATTCACA-CC-CCT
-TAG-AGGTAT-A-TAGCAATTGT-GTGTGAGTATCGT-GTC-CTATTGTCAGCC-CC-
-TAG-AGGTAT-A-TAGCCATTGT-GTGTGAGGATCGT-GTC-GTATTGTCTGCCAC--
-TAG-AAGT-TCA-TAGCCGTTGT-GTGTGAGGAT-GTGGTC-GTGCTGTCTGGTAC--
-TGG-AAGT-GCAGC-GCGGGTGT-TTGGGGGGATAA-GGGC-TTGCTGACTTGTCC--
-TTG-AACT-GTAGC-GCGGTTGTATT-AGGGGACAACAG-C-TTGTTGACTTG-CCA-
GTTG--ACT-GTAGCTGC-TTTGCGTC-AGGGGCCTACTG-C-TTATTGAATAGCCC--
GTCG--ACA-GCAGCTGC-TTCGCGTC-GGGGGTCTAC-GACAGT-TTGAACA-CCCA-
Score of aligned sequences = 2010
The aligned sequences are saved in output file. Printing them here as well:
C-TA-GATAGGAA-C-CGTC--C-TCTT-A-CC-G-GC-GCCG-TAC-ATTGCCCTTTGGTAA--GTCCAAC-CATTT---GA-GTC-CA--AGCGGG-T-T-TACCAGAGGACAAGAATTTC-GTT-C
A-TT-GACAGAAACC-CG-C--G-GC-G-ACCC-G-GC-CCCG-CGC-CG

TTCCCC-G--AAAG-GTC---CTTCAG-C-TTA---AGCC-GC-GGTAA-GC-GGAA-GAGGTT-A-TACG-ATCTG---GAGCAGGTAGCACTCTC--GAC--TT-AAA-A--G-CTCTTTCGC-CTCAGG---GA-AGT-CGTTAGCCTGC-C-CAATCTA-A-TGG---GTTA--G-GCACCTTG-G-ATC-CTGA-TTAAG-CATTCAA-CATA---C-TACGCGCCCATCT-AAACGAT-ATCGT--ACAAGGG-GCC-TGGGACGG--CA---ACCAA-CTCAG--A-T-TGCCA-TATCTCGCA--AATA--T-AAAT-CTAAGCT-GG-TGA-TCC-CATCGG-GTTGATTCG-AATTCTCGTGTCAAGAAG-ATGTCCA---
TTCCCT-G--AAAG-GTC---CATAAG-C-TTA---AGAC-GC-TGTAA-ACAGG-A-GAGGTT-A-TACC-ATCAG---GAGCAGTTAGC-C-CACGGGACA-TGAAAACAT-G-TTCTTTAG---T--GG---GA-TGT-CGGTAGGCTGC-CGCAA-CTA-AGT-G---GTTA--T-GCACC-TGCG-ATC-CCGA-TTTAG-CAATGAAGC-GA---G-TTCGCGCCCATTT-AAACGAT-ATTGT--ACGA-GCAGCC-TGCGAGGG--GA---TCTAC-CTCGG--A-T-TGTCA-TATCTAGCA--AATA-GT-AAA--CTAATTT-TG-TG--CCCGCATCGG-GTTGCTTCGTAATCCT-GGGTCAAGAAG-GTGT-CT--T
TTCGTT-G--TCAG-G-CG--CATAAG-C-CGAA--AG-C-GCGTG-AA-ATTGG-A-GCGG-C-AGGACC-ATCAG---GCGCAGTAAGC-C-CACGGGCCA-TGTAACCCG-G-GTCTTTAG---T--GG---GC-TAT-CTGTAGGCTGA-TGTAA-ATAGA-T--T--GTTT--T-GAACC-CACG-ATC-CGGATTTTA--CAGTGAA-C-

-GTTTCGCA-T-C-CCGAATA-GAGA-GCAGGA-AC-CACGGCG-CACA-GC-CCGGAG-TCT-G-GCACG-AGCCGCGGT-TGG--GCCC-A-TCG-GCGTCTAATT-G--GGC-ATAGG--C-AGGC-GC-GTTC-CATGTA-TC-C----GGGAAACT-AG-GAGAG-ACCCATGCCTGTGCCACTTG-TA---A--CCGGGCGCAT-TCTGA--CGTCCCAGGATCACTAAT--AGCGACGTC-T--TC-GATA-CATCCG-AA--TCATCT--AACCA--C-CTGT-TCTTCG-TGGAAGTGG-GCTTATT-GG-CG-AA--T-AT-TTGTG--CTT---GTGAGCCAGGCTA-AAG--GTAGTC-TC-AGCCT-CG---AA-ACGCTGA-A-AAACTC--AT-GAG-C---AGAAATC--AGCGC-CCGTCGGCGATC--TG-G--GTCA-ATCAACT--CCACTC-CCGCATCGAGGATCAATGCC-GGA-GCCCT-CT-CATGTTT
-GTTACGCA-T-C-CAGAACATGGGA--CAGGA-AC-TACAGCA-CAGA-GC-CCGGTG-GCTCG-AC-CG-AGGCTCGG--TGG-ACCCC-A-CCG-GTGTATAA---GAGGGC-ATA--CAC-AGCC-GC-GTTCTC-TGT-GT-TC-T--GGG-AACC-CG-CTGAG-ACCCATTCCTCTGCCACTT-TTA---AC-CCATAC-TAT-TCTGA--CGCCCCAAGATCAATAA--AACCGATG-CCT-ATC--ATA-CATCCG-AA--TGGT-T-TAA-CA-AC-CTGT-TCTT-GATAGAATTGG-GCCTA-GCGG-CGAAA-GT-ATATT-CG--C-----CTGAGTCAGGCCA-AAG--GTAGTCTTC--G-CTTC---TAA-CCTCTG--ACAAACTG--GT-GAG-C---ATAAA-CC-ATCAC-CAGTCTGCG--CGCTG-G--GTCA-ATCAACT--CCCCTC-CCGAATCGAGGA

-AGA--CAC-GTC-C-CA-GCCGTG---C-CTCA-CAGACTGGACG-CACTA-G-AAA-GCGCGTGGTAGCCCAATAGT-C-C----G-GTCG-TTG-AT-CTT-ACTAGTTAAG-T-GGGTCCTT-GATG-TGCTC-AA-CCATGGTA-TAGTCAC-GTGAGAT--ACTTTCA--GTG-CTCA-GGT-CTGCAT-CCTC-A-CCTCAGG-GA-ACG-GAAGC-TAAG-G-G-GAC-CGG-AGTCCGAA-T-TGGAGC-C-GAAG-G-A-T--CAGACG--GAGGCGA-TCC-T-GAAGTTTGCACTCCGACG-G-AGTTA--ATAGA-GCTTGTAC-GCCCCGA-CGTGCCTAG-GCCA-TG-TCGAATCC--CAGGGCT---ATTCTC--TAACGGGAG-C-G-CTC--ACCC-ACAACG-GCC--TCTATG-AGC-CGGTGATAAGGGGAA---ACA-AAAG-TGGCGTTATGA-GACGAGT---GT--GGA-GTTGCAGAT-GTA-AA-CAGCGTACCCCCG--GATTC--GAA----C-T-CCCGAC-GCTG-GTA--TGCG-AC-GACTC-A-CTT-GGC-TTTTTGA-ACT-C-GCCAT--AGGGAAAACGTT---AG-TGTGATCAA--ATTCC
-AGA--TACT-TC-CT-A-TCCGCG---A-CACA-CAGACTGGGCG-CACCA-G-GAA-GCGCGTGGTAGACCAAAAGT-C-C----G-GTCG-TTGT-T-CTT-ACTACTTAAG-A-GGGGCATTA-ATGC-GCT--AACCCATGGTA-TTGACAC-GTGAGCTGC-C-TTCA--GAG-CTCAC-GT-CTGCAT-CCTCT--CCTCAGA-TA-A-GTGAAGC-TAAG-G-G-CTC-CGG-AG-GCAAATT-TAGAGC-C-GAAT-G-ATT--C-TACG--AAGGCGA-TCC-T-GAAGTTTGCAGTAGGTTG-G-AGTTACG--AGG-GCTGGAAC-GCCCCCA-GGTGCCTAG-G

10-element Vector{Any}:
 0.211510075
 0.284566949
 0.285044531
 0.374034991
 0.636511294
 0.740590928
 0.928163685
 1.202157868
 1.409008206
 1.573944072

In [91]:
# get GA times

GA_times = [];

f = open("GAtimes.txt")
f = readlines(f)

for str in f
    str = parse(Float64, str)
    push!(GA_times, str)
end

In [92]:
GA_times

10-element Vector{Any}:
 0.564
 1.117
 1.503
 2.059
 2.49
 2.991
 3.552
 4.272
 5.091
 5.27

In [93]:
function plot_time_ACO_vs_GA()
    x = collect(50:50:500)
    # different length sequences
    plot(x, [ACO_times, GA_times],
        title = "ACO times vs GA times for 10 sequences", 
        label = ["ACO times" "GA times"]
    )
    xlabel!("Length of Sequence")
    ylabel!("Times")
    savefig("plot_time_ACO_vs_GA.png")
end

plot_time_ACO_vs_GA (generic function with 1 method)

In [94]:
plot_time_ACO_vs_GA()